In [2]:
# %pip install speechrecognition
# %pip install pyttsx3
# %pip install vosk
# %pip install sounddevice


  Using cached sounddevice-0.4.4-py3-none-win_amd64.whl (195 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# %pip install pipwin
# !pipwin install pyaudio


In [8]:
import pyttsx3

tts = pyttsx3.init()
rate = tts.getProperty('rate')  # Скорость произношения
tts.setProperty('rate', rate-40)

volume = tts.getProperty('volume')  # Громкость голоса
tts.setProperty('volume', volume+0.9)

voices = tts.getProperty('voices')
# for voice in voices:
#     print("Voice:", voice.name)

# Задать голос по умолчанию
tts.setProperty('voice', 'ru')

# Попробовать установить предпочтительный голос
# for voice in voices:
#     if voice.name == 'Anna':
#         tts.setProperty('voice', voice.id)


In [4]:
import argparse
import os
import queue
import sounddevice as sd
import vosk
import sys

# print(sd.query_devices())
device_info = sd.query_devices(15, 'input')
# soundfile expects an int, sounddevice provides a float:
samplerate = int(device_info['default_samplerate'])
print(samplerate)

q = queue.Queue()

def callback(indata, frames, time, status):
    """This is called (from a separate thread) for each audio block."""
    if status:
        print(status, file=sys.stderr)
    q.put(bytes(indata))


# model = vosk.Model("C:\\PycharmProjects\\Speech2Text\\vosk-model-ru-0.22")
print("Try load model")
model = vosk.Model("C:\\PycharmProjects\\Speech2Text\\vosk-model-small-ru-0.22")
print("Loaded model:", model)

with sd.RawInputStream(samplerate, blocksize=8000, device=15, dtype='int16', channels=1, callback=callback):
    print('#' * 80)
    print('Press Ctrl+C to stop the recording')
    print('#' * 80)

    rec = vosk.KaldiRecognizer(model, 8000)
    while True:
        data = q.get()
        print("Debug", data)
        if rec.AcceptWaveform(data):
            print(rec.Result())
        else:
            print(rec.PartialResult())
        # if dump_fn is not None:
        #     dump_fn.write(data)


48000
Try load model
Loaded model: <vosk.Model object at 0x000001B2A1A99DE0>
################################################################################
Press Ctrl+C to stop the recording
################################################################################
Debug b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

Exception: Failed to process waveform

In [5]:
from vosk import Model, KaldiRecognizer
import os
import pyaudio

print("Try load model")
model = Model(r"C:\\PycharmProjects\\Speech2Text\\vosk-model-small-ru-0.22") # полный путь к модели
print("Model loaded", model)

rec = KaldiRecognizer(model, 8000)
p = pyaudio.PyAudio()
stream = p.open(
    format=pyaudio.paInt16, 
    channels=1, 
    rate=8000, 
    input=True, 
    frames_per_buffer=8000
)
stream.start_stream()

while True:
    data = stream.read(4000)
    if len(data) == 0:
        break

    print(rec.Result() if rec.AcceptWaveform(data) else rec.PartialResult())

print(rec.FinalResult())

Try load model
Model loaded <vosk.Model object at 0x000001B2C7622260>


Exception: Failed to process waveform

In [ ]:
import speech_recognition as sr

r = sr.Recognizer()

while True:
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source, duration=0.3)

        print("Speak")
        audio = r.listen(source)

        try:
            text = r.recognize_google(audio, language="ru-RU")
            print("Speaker:", text)
            tts.say(text)
            tts.runAndWait()
        except:
            print("Please say again")
